# Comparing the 2023 and 2024 Michelin Guide to France

In [1]:
import pandas as pd

In [2]:
france_23 = pd.read_csv('../../../2023/data/France/all_restaurants(arrondissements).csv')
france_24 = pd.read_csv('../../data/France/all_restaurants(arrondissements).csv')

In [3]:
france_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1033 non-null   object 
 1   address         1033 non-null   object 
 2   location        1033 non-null   object 
 3   arrondissement  1033 non-null   object 
 4   department_num  1033 non-null   object 
 5   department      1033 non-null   object 
 6   capital         1033 non-null   object 
 7   region          1033 non-null   object 
 8   price           1033 non-null   object 
 9   cuisine         1033 non-null   object 
 10  url             989 non-null    object 
 11  award           1033 non-null   object 
 12  stars           1033 non-null   float64
 13  longitude       1033 non-null   float64
 14  latitude        1033 non-null   float64
dtypes: float64(3), object(12)
memory usage: 121.2+ KB


In [4]:
france_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1017 non-null   object 
 1   address         1017 non-null   object 
 2   location        1017 non-null   object 
 3   arrondissement  1017 non-null   object 
 4   department_num  1017 non-null   object 
 5   department      1017 non-null   object 
 6   capital         1017 non-null   object 
 7   region          1017 non-null   object 
 8   price           1017 non-null   object 
 9   cuisine         1017 non-null   object 
 10  url             973 non-null    object 
 11  award           1017 non-null   object 
 12  stars           1017 non-null   float64
 13  longitude       1017 non-null   float64
 14  latitude        1017 non-null   float64
dtypes: float64(3), object(12)
memory usage: 119.3+ KB


----
&nbsp;
## Identify new or removed entries focusing on 1+ star restaurants

In [5]:
# Filter france_23 and france_24 for star ratings >= 1
subset_23 = france_23[france_23['stars'] >= 1][['name', 'address', 'location', 'stars']]
subset_24 = france_24[france_24['stars'] >= 1][['name', 'address', 'location', 'stars']]

# Turn the filtered data into sets of tuples
set_subset_23 = set(tuple(x) for x in subset_23.values)
set_subset_24 = set(tuple(x) for x in subset_24.values)

# Find new and closed Michelin stars
potential_new_stars = set_subset_24 - set_subset_23
potential_closed_stars = set_subset_23 - set_subset_24

In [6]:
print(f'Potentially entered the guide:\n{len(potential_new_stars)} starred restaurants in 2024 guide not in 2023')
print(f'\nPotentially left the guide:\n{len(potential_closed_stars)} starred restaurants in 2023 guide not in 2024')

Potentially entered the guide:
125 starred restaurants in 2024 guide not in 2023

Potentially left the guide:
113 starred restaurants in 2023 guide not in 2024


In [7]:
# Now find name changes. For this, we need to find common addresses with different names
common_locations = set(subset_23['location']).intersection(set(subset_24['location']))

# Filtering out the common addresses from both DataFrames
common_23 = subset_23[subset_23['location'].isin(common_locations)]
common_24 = subset_24[subset_24['location'].isin(common_locations)]

In [8]:
# Finding the entries with the same address and stars but different names
name_changes = []
for index, row in common_24.iterrows():
    # Filter the common_23 DataFrame to get rows with the same address and stars as the current row from common_24
    matching_23 = common_23[(common_23['address'] == row['address']) & (common_23['location'] == row['location']) & (common_23['stars'] == row['stars'])]
    # Iterate over the filtered DataFrame to compare names
    for _, match in matching_23.iterrows():
        if match['name'] != row['name']:
            name_changes.append((match['name'], row['name'], row['address'], row['location'], row['stars']))

name_changes_df = pd.DataFrame(name_changes, columns=['old_name', 'new_name', 'address', 'location', 'stars'])

There are some trivial differences in `name` between 2023 and 2024

In [9]:
name_changes_df

,old_name,new_name,address,location,stars
0,Régis et Jacques Marcon,Restaurant Marcon,Larsiallas,"Saint-Bonnet-le-Froid, 43290",3.0
1,Yoann Conte,La Table de Yoann Conte,13 vieille route des Pensières,"Veyrier-du-Lac, 74290",2.0
2,La Pyramide - Patrick Henriroux,La Pyramide - Maison Henriroux,14 boulevard Fernand-Point,"Vienne, 38200",2.0
3,Pastis,Pastis Restaurant,3 rue Terral,"Montpellier, 34000",1.0
4,La Maison Caillet,Maison Caillet,22 rue André-Fiquet,"Valmont, 76540",1.0
5,Entre Vigne et Garrigue,Maison Chenet - Entre Vigne et Garrigue,600 route de Saint-Bruno,"Pujaut, 30131",1.0
6,Leclere,Leclère,8 rue André-Michel,"Montpellier, 34000",1.0
7,Auberge Saint-Laurent,Auberge St-Laurent,1 rue de la Fontaine,"Sierentz, 68510",1.0
8,Girardin - Gastronomique,Restaurant Girardin,19 rue des Têtes,"Colmar, 68000",1.0
9,L'Orangerie,Le George,"Four Seasons George V, 31 avenue George-V","Paris, 75008",1.0


In [10]:
from fuzzywuzzy import fuzz

# Function to preprocess and compare names
def preprocess(text):
    # Basic preprocessing: lowercasing and stripping spaces
    return text.lower().strip()

def compare_names(name1, name2):
    name1 = preprocess(name1)
    name2 = preprocess(name2)
    return fuzz.token_sort_ratio(name1, name2)  # Comparing with sorted tokens to handle reordering

# Applying the comparison across the DataFrame rows
name_changes_df['similarity_score'] = name_changes_df.apply(lambda x: compare_names(x['old_name'], x['new_name']), axis=1)

In [11]:
name_changes_df

,old_name,new_name,address,location,stars,similarity_score
0,Régis et Jacques Marcon,Restaurant Marcon,Larsiallas,"Saint-Bonnet-le-Froid, 43290",3.0,46
1,Yoann Conte,La Table de Yoann Conte,13 vieille route des Pensières,"Veyrier-du-Lac, 74290",2.0,65
2,La Pyramide - Patrick Henriroux,La Pyramide - Maison Henriroux,14 boulevard Fernand-Point,"Vienne, 38200",2.0,84
3,Pastis,Pastis Restaurant,3 rue Terral,"Montpellier, 34000",1.0,52
4,La Maison Caillet,Maison Caillet,22 rue André-Fiquet,"Valmont, 76540",1.0,90
5,Entre Vigne et Garrigue,Maison Chenet - Entre Vigne et Garrigue,600 route de Saint-Bruno,"Pujaut, 30131",1.0,77
6,Leclere,Leclère,8 rue André-Michel,"Montpellier, 34000",1.0,92
7,Auberge Saint-Laurent,Auberge St-Laurent,1 rue de la Fontaine,"Sierentz, 68510",1.0,92
8,Girardin - Gastronomique,Restaurant Girardin,19 rue des Têtes,"Colmar, 68000",1.0,44
9,L'Orangerie,Le George,"Four Seasons George V, 31 avenue George-V","Paris, 75008",1.0,50


In [12]:
# Filter rows where the similarity score is greater than 70
high_similarity_rows = name_changes_df[name_changes_df['similarity_score'] > 70]

In [13]:
# Iterate over these rows to update the names in france_23
for _, row in high_similarity_rows.iterrows():
    # Update france_23 where the address and location match
    france_23.loc[(france_23['location'] == row['location']), 'name'] = row['new_name']

---
&nbsp;
## New ⭐⭐⭐ Restaurants, New ⭐⭐ Restaurants

In [14]:
# Identify New Two-Star and Three-Star Restaurants
# Filter france_23 and france_24 for star ratings >= 1
two_and_three_stars_23 = france_23[france_23['stars'] >= 2][['name', 'address', 'location', 'stars']]
two_and_three_stars_24 = france_24[france_24['stars'] >= 2][['name', 'address', 'location', 'stars']]

# Convert them to sets of (address, stars) tuples
set_two_and_three_stars_23 = set(tuple(x) for x in two_and_three_stars_23[['name', 'address', 'location', 'stars']].values)
set_two_and_three_stars_24 = set(tuple(x) for x in two_and_three_stars_24[['name', 'address', 'location', 'stars']].values)

# Find the difference to identify new two-star and three-star restaurants
new_two_and_three_stars = set_two_and_three_stars_24 - set_two_and_three_stars_23

In [15]:
# Create a DataFrame from the dictionary
df = pd.DataFrame(new_two_and_three_stars, columns=['name', 'address', 'location', 'stars'])

# Now, let's sort the DataFrame first by stars in descending order, then by postal code in ascending order
sorted_df = df.sort_values(by=['stars'], ascending=[False])

# Display the sorted DataFrame
sorted_df[['name', 'address', 'location', 'stars']]

,name,address,location,stars
3,Restaurant Marcon,Larsiallas,"Saint-Bonnet-le-Froid, 43290",3.0
4,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","Paris, 75008",3.0
13,La Table du Castellet,"3001 route des Hauts-du-Camp, au Circuit Paul ...","Le Castellet, 83330",3.0
0,Casadelmar,route de Palombaggia,"Porto-Vecchio, 20137",2.0
1,Le Pressoir d'Argent - Gordon Ramsay,"InterContinental Bordeaux - Le Grand Hôtel, 2-...","Bordeaux, 33000",2.0
2,La Table de Yoann Conte,13 vieille route des Pensières,"Veyrier-du-Lac, 74290",2.0
5,Le Coquillage,Lieu-dit Le Buot,"Saint-Méloir-des-Ondes, 35350",2.0
6,L'Orangerie,"Four Seasons George V, 31 avenue George-V","Paris, 75008",2.0
7,La Table des Amis by Christophe Bacquié,"Les Mas Les Eydins, 2420 chemin du Four","Bonnieux, 84480",2.0
8,Maison Benoît Vidal,"Sur-les-Bois, 79 route de Thônes","Annecy, 74940",2.0


It was announced in the press that there are two new 3* and eight new 2*. We check for any changes

In [63]:
# Initialize a list to store the changes
changes = []

# Loop through the new restaurants DataFrame
for _, new_row in sorted_df.iterrows():
    # Look for a match in the previous year's data
    matching_old_rows = subset_23[
        (subset_23['name'] == new_row['name']) &
        #(subset_23['address'] == new_row['address']) & 
        (subset_23['location'] == new_row['location'])
    ]
    
    # If there's a match, check for changes
    for _, old_row in matching_old_rows.iterrows():
        # If any detail has changed, add the comparison to the changes list
        if not (old_row['stars'] == new_row['stars']):
            changes.append({
                'old_name': old_row['name'],
                'new_name': new_row['name'],
                'old_address': old_row['address'],
                'new_address': new_row['address'],
                'old_location': old_row['location'],
                'new_location': new_row['location'],
                'old_stars': old_row['stars'],
                'new_stars': new_row['stars']
            })

# Convert the changes list to a DataFrame
changes_df = pd.DataFrame(changes)

In [64]:
changes_df

,old_name,new_name,old_address,new_address,old_location,new_location,old_stars,new_stars
0,Le Gabriel - La Réserve Paris,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","La Réserve Paris, 42 avenue Gabriel","Paris, 75008","Paris, 75008",2.0,3.0
1,Sylvestre Wahid - Les Grandes Alpes,Sylvestre Wahid - Les Grandes Alpes,28 rue de l'Église,"28 rue de l'Église, Courchevel 1850","Courchevel, 73120","Courchevel, 73120",1.0,2.0
2,Maison Ronan Kervarrec,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760","Saint-Grégoire, 35760",1.0,2.0
3,René et Maxime Meilleur,René et Maxime Meilleur,Hameau de Saint-Marcel,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440","Saint-Martin-de-Belleville, 73440",3.0,2.0
4,Le Jules Verne,Le Jules Verne,Tour Eiffel - Avenue Gustave-Eiffel,Tour Eiffel - Avenue Gustave-Eiffel,"Paris, 75007","Paris, 75007",1.0,2.0
5,Maison Ruggieri,Maison Ruggieri,11 rue Treilhard,11 rue Treilhard,"Paris, 75008","Paris, 75008",1.0,2.0
6,L'Orangerie,L'Orangerie,"Four Seasons George V, 31 avenue George-V","Four Seasons George V, 31 avenue George-V","Paris, 75008","Paris, 75008",1.0,2.0


In [54]:
import difflib

# Function to check name similarity
def is_similar(name1, name2, threshold=0.8):
    return difflib.SequenceMatcher(None, name1, name2).ratio() > threshold

changes = []

for _, new_row in sorted_df.iterrows():
    # Look for a match in the previous year's data based on location
    matching_old_rows = subset_23[subset_23['location'] == new_row['location']]
    
    # Check for similar names and if there's a change in star rating
    for _, old_row in matching_old_rows.iterrows():
        if is_similar(old_row['name'], new_row['name']):
            # If star ratings are different, record the change
            if old_row['stars'] != new_row['stars']:
                changes.append({
                    'old_name': old_row['name'],
                    'new_name': new_row['name'],
                    'old_address': old_row['address'],
                    'new_address': new_row['address'],
                    'old_location': old_row['location'],
                    'new_location': new_row['location'],
                    'old_stars': old_row['stars'],
                    'new_stars': new_row['stars']
                })

# Convert the changes list to a DataFrame
changes_df = pd.DataFrame(changes)

In [55]:
changes_df

,old_name,new_name,old_address,new_address,old_location,new_location,old_stars,new_stars
0,Le Gabriel - La Réserve Paris,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","La Réserve Paris, 42 avenue Gabriel","Paris, 75008","Paris, 75008",2.0,3.0
1,Sylvestre Wahid - Les Grandes Alpes,Sylvestre Wahid - Les Grandes Alpes,28 rue de l'Église,"28 rue de l'Église, Courchevel 1850","Courchevel, 73120","Courchevel, 73120",1.0,2.0
2,Maison Ronan Kervarrec,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760","Saint-Grégoire, 35760",1.0,2.0
3,René et Maxime Meilleur,René et Maxime Meilleur,Hameau de Saint-Marcel,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440","Saint-Martin-de-Belleville, 73440",3.0,2.0
4,Le Jules Verne,Le Jules Verne,Tour Eiffel - Avenue Gustave-Eiffel,Tour Eiffel - Avenue Gustave-Eiffel,"Paris, 75007","Paris, 75007",1.0,2.0
5,Maison Ruggieri,Maison Ruggieri,11 rue Treilhard,11 rue Treilhard,"Paris, 75008","Paris, 75008",1.0,2.0
6,L'Orangerie,L'Orangerie,"Four Seasons George V, 31 avenue George-V","Four Seasons George V, 31 avenue George-V","Paris, 75008","Paris, 75008",1.0,2.0


----
&nbsp;
## Merging dfs for comparison

In [11]:
# Merge using the new 'fuzzy_matched_name' in france_23 and 'name' in france_24
comparison_df = france_23.merge(
    france_24, 
    left_on='fuzzy_matched_name',
    right_on='name',
    suffixes=('_23', '_24'), 
    how='outer', 
    indicator=True
)

In [14]:
# Remove unwanted columns
columns_to_drop = ['department_num_23', 'department_23', 'capital_23', 'price_23', 'cuisine_23', 'latitude_23', 'longitude_23', 
                   'department_num_24', 'department_24', 'capital_24', 'price_24', 'cuisine_24', 'latitude_24', 'longitude_24']

comparison_df.drop(columns=columns_to_drop, inplace=True)

In [16]:
# Restaurants promoted to a star rating in 2024
promotions = comparison_df[(comparison_df['stars_23'].isna() | (comparison_df['stars_23'] < 1)) & (comparison_df['stars_24'] >= 1)]

# Restaurants demoted from a star rating after 2023
demotions = comparison_df[(comparison_df['stars_23'] >= 1) & (comparison_df['stars_24'].isna() | (comparison_df['stars_24'] < 1))]

In [28]:
promotions

In [33]:
demotions

,name,address,location,stars
4,Restaurant Marcon,Larsiallas,"Saint-Bonnet-le-Froid, 43290",3.0
15,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","Paris, 75008",3.0
0,La Table du Castellet,"3001 route des Hauts-du-Camp, au Circuit Paul ...","Le Castellet, 83330",3.0
10,Casadelmar,route de Palombaggia,"Porto-Vecchio, 20137",2.0
5,Le Pressoir d'Argent - Gordon Ramsay,"InterContinental Bordeaux - Le Grand Hôtel, 2-...","Bordeaux, 33000",2.0
7,Le Coquillage,Lieu-dit Le Buot,"Saint-Méloir-des-Ondes, 35350",2.0
12,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760",2.0
1,La Pyramide - Maison Henriroux,14 boulevard Fernand-Point,"Vienne, 38200",2.0
13,Sylvestre Wahid - Les Grandes Alpes,"28 rue de l'Église, Courchevel 1850","Courchevel, 73120",2.0
2,René et Maxime Meilleur,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440",2.0


In [32]:
# New entries are those that only appear in the 2024 data
new_entries = comparison_df[(comparison_df['_merge'] == 'right_only')]

# Closures are those that only appear in the 2023 data
closures = comparison_df[(comparison_df['_merge'] == 'left_only')]

,name,address,location,stars,city,postal_code
4,Restaurant Marcon,Larsiallas,"Saint-Bonnet-le-Froid, 43290",3.0,Saint-Bonnet-le-Froid,43290
15,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","Paris, 75008",3.0,Paris,75008
0,La Table du Castellet,"3001 route des Hauts-du-Camp, au Circuit Paul ...","Le Castellet, 83330",3.0,Le Castellet,83330
10,Casadelmar,route de Palombaggia,"Porto-Vecchio, 20137",2.0,Porto-Vecchio,20137
5,Le Pressoir d'Argent - Gordon Ramsay,"InterContinental Bordeaux - Le Grand Hôtel, 2-...","Bordeaux, 33000",2.0,Bordeaux,33000
7,Le Coquillage,Lieu-dit Le Buot,"Saint-Méloir-des-Ondes, 35350",2.0,Saint-Méloir-des-Ondes,35350
12,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760",2.0,Saint-Grégoire,35760
1,La Pyramide - Maison Henriroux,14 boulevard Fernand-Point,"Vienne, 38200",2.0,Vienne,38200
13,Sylvestre Wahid - Les Grandes Alpes,"28 rue de l'Église, Courchevel 1850","Courchevel, 73120",2.0,Courchevel,73120
2,René et Maxime Meilleur,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440",2.0,Saint-Martin-de-Belleville,73440


In [26]:
new_entries

In [27]:
closures

71

In [36]:
# Restaurants that gained stars
gained_stars = comparison_df[(comparison_df['stars_23'] < comparison_df['stars_24'])]

# Restaurants that lost stars
lost_stars = comparison_df[(comparison_df['stars_23'] > comparison_df['stars_24'])]

In [37]:
gained_stars

,name,address_23,location_23,arrondissement_23,department_num_23,department_23,capital_23,region_23,price_23,cuisine_23,...,capital_24,region_24,price_24,cuisine_24,url_24,award_24,stars_24,longitude_24,latitude_24,_merge
49,Relais de la Poste,21 rue du Général-de-Gaulle,"La Wantzenau, 67610",Strasbourg,67,Bas-Rhin,Strasbourg,Grand Est,€€€,Modern Cuisine,...,Mont-de-Marsan,Nouvelle-Aquitaine,€€€€,Classic Cuisine,https://www.relaisposte.com,2 Stars,2.0,-1.217965,43.778734,both
60,Le Gabriel - La Réserve Paris,"La Réserve Paris, 42 avenue Gabriel","Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,Creative,...,Paris,Île-de-France,€€€€,Creative,https://www.lareserve-paris.com/,3 Stars,3.0,2.313208,48.869731,both
83,Racine,1 rue du Chapeau-Rouge,"Lyon, 69009",Lyon,69,Rhône,Lyon,Auvergne-Rhône-Alpes,€,Modern Cuisine,...,Châlons-en-Champagne,Grand Est,€€€€,Creative,https://www.racine.re/,2 Stars,2.0,4.037580,49.252864,both
121,Maison Ruggieri,11 rue Treilhard,"Paris, 75008",8th (Élysée),75,Paris,Paris,Île-de-France,€€€€,"Modern Cuisine, Creative",...,Paris,Île-de-France,€€€€,"Modern Cuisine, Creative",http://www.maisonruggieri.fr,2 Stars,2.0,2.314134,48.877027,both
152,Rouge,2 rue de Foresta,"Nice, 06000",Nice,06,Alpes-Maritimes,Nice,Provence-Alpes-Côte d'Azur,€,"Mediterranean Cuisine, Sharing",...,Nîmes,Occitanie,€€€€,Creative,http://www.margaret-hotelchouleur.com,1 Star,1.0,4.357880,43.837170,both
179,Maison Ronan Kervarrec,1 impasse du Vieux-Bourg,"Saint-Grégoire, 35760",Rennes,35,Ille-et-Vilaine,Rennes,Brittany,€€€€,Modern Cuisine,...,Rennes,Brittany,€€€€,Modern Cuisine,http://www.le-saison.com/fr,2 Stars,2.0,-1.680556,48.153301,both
213,La Chaumière,96 avenue Emmanuel-Brousse,"Font-Romeu, 66120",Prades,66,Pyrénées-Orientales,Perpignan,Occitanie,€€,Catalan,...,Lons-le-Saunier,Bourgogne-Franche-Comté,€€€€,Creative,https://www.lachaumiere-dole.fr/fr/,1 Star,1.0,5.495128,47.065097,both
215,La Chaumière,2 rue Paul-Lasnier,"Aubigny-sur-Nère, 18700",Vierzon,18,Cher,Bourges,Centre-Val de Loire,€€,Traditional Cuisine,...,Lons-le-Saunier,Bourgogne-Franche-Comté,€€€€,Creative,https://www.lachaumiere-dole.fr/fr/,1 Star,1.0,5.495128,47.065097,both
254,Racines,8 passage des Bains,"Annecy, 74000",Annecy,74,Haute-Savoie,Annecy,Auvergne-Rhône-Alpes,€€,Modern Cuisine,...,Rennes,Brittany,€€€,Modern Cuisine,https://www.racines-restaurant.fr/,1 Star,1.0,-1.682965,48.106943,both
258,Racines,46 boulevard de l'Impératrice,"Étaples, 62630",Montreuil,62,Pas-de-Calais,Arras,Hauts-de-France,€€,Modern Cuisine,...,Rennes,Brittany,€€€,Modern Cuisine,https://www.racines-restaurant.fr/,1 Star,1.0,-1.682965,48.106943,both


In [38]:
lost_stars

,name,address_23,location_23,arrondissement_23,department_num_23,department_23,capital_23,region_23,price_23,cuisine_23,...,capital_24,region_24,price_24,cuisine_24,url_24,award_24,stars_24,longitude_24,latitude_24,_merge
1,René et Maxime Meilleur,Hameau de Saint-Marcel,"Saint-Martin-de-Belleville, 73440",Albertville,73,Savoie,Chambéry,Auvergne-Rhône-Alpes,€€€€,"Creative, Regional Cuisine",...,Chambéry,Auvergne-Rhône-Alpes,€€€€,"Creative, Regional Cuisine",https://www.la-bouitte.com/fr/,2 Stars,2.0,6.513306,45.369046,both
31,Au 14 Février,Le Plâtre-Durand,"Saint-Amour-Bellevue, 71570",Mâcon,71,Saône-et-Loire,Mâcon,Bourgogne-Franche-Comté,€€€€,Creative,...,Châteauroux,Centre-Val de Loire,€€€€,Modern Cuisine,https://www.sv-au14fevrier.com/,1 Star,1.0,1.865023,46.951707,both
32,Au 14 Février,Le Plâtre-Durand,"Saint-Amour-Bellevue, 71570",Mâcon,71,Saône-et-Loire,Mâcon,Bourgogne-Franche-Comté,€€€€,Creative,...,Lyon,Auvergne-Rhône-Alpes,€€€€,Creative,https://www.ly-au14fevrier.com/,1 Star,1.0,4.826382,45.762221,both
73,Auberge du Cheval Blanc,4 rue de Wissembourg,"Lembach, 67510",Haguenau-Wissembourg,67,Bas-Rhin,Strasbourg,Grand Est,€€€€,Creative,...,Strasbourg,Grand Est,€€€€,Modern Cuisine,https://www.cheval-blanc-lembach.fr/fr/,1 Star,1.0,7.790658,49.003325,both
82,Racine,6 place Godinot,"Reims, 51100",Reims,51,Marne,Châlons-en-Champagne,Grand Est,€€€€,Creative,...,Lyon,Auvergne-Rhône-Alpes,€,Modern Cuisine,https://www.racinerestaurant-lyon.com/,Bib Gourmand,0.5,4.807470,45.774020,both
212,La Chaumière,346 avenue du Maréchal-Juin,"Dole, 39100",Dole,39,Jura,Lons-le-Saunier,Bourgogne-Franche-Comté,€€€€,Creative,...,Perpignan,Occitanie,€€,Catalan,https://www.restaurantlachaumiere.fr/,Bib Gourmand,0.5,2.043353,42.505297,both
251,Racines,4 passage Antoinette Caillot,"Rennes, 35000",Rennes,35,Ille-et-Vilaine,Rennes,Brittany,€€€,Modern Cuisine,...,Bourg-en-Bresse,Auvergne-Rhône-Alpes,€€,Modern Cuisine,https://domainedulac-racines.fr/,Bib Gourmand,0.5,5.181661,46.197984,both
252,Racines,4 passage Antoinette Caillot,"Rennes, 35000",Rennes,35,Ille-et-Vilaine,Rennes,Brittany,€€€,Modern Cuisine,...,Arras,Hauts-de-France,€€,Modern Cuisine,https://restaurant-racines.fr/,Bib Gourmand,0.5,1.635408,50.514614,both
253,Racines,4 passage Antoinette Caillot,"Rennes, 35000",Rennes,35,Ille-et-Vilaine,Rennes,Brittany,€€€,Modern Cuisine,...,Annecy,Auvergne-Rhône-Alpes,€€,Modern Cuisine,https://racines-annecy.fr,Bib Gourmand,0.5,6.123207,45.899492,both
319,Auberge de l'Abbaye,47 place des Anciens-Combattants,"Ambronay, 01500",Belley,01,Ain,Bourg-en-Bresse,Auvergne-Rhône-Alpes,€€€,Modern Cuisine,...,Saint-Lô,Normandy,€€,Modern Cuisine,https://www.aubergedelabbayehambye.com/,Bib Gourmand,0.5,-1.267777,48.924306,both
